In [ ]:
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

In [ ]:
from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']